In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch
import torch.nn as nn
from torchvision import utils
def visTensor(tensor, ch=0, allkernels=False, nrow=8, padding=1): 
    n,c,w,h = tensor.shape
    if allkernels: tensor = tensor.view(n*c, -1, w, h)
    elif c != 3: tensor = tensor[:,ch,:,:].unsqueeze(dim=1)
    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    plt.figure( figsize=(nrow,rows) )
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

pathModel = "models/densenet_100epochs_data_norm_randomCropRotation.pt"

model = torchvision.models.densenet121(pretrained=False)
num_ftrs = model.classifier.in_features
model.classifier  = nn.Sequential(nn.Linear(num_ftrs, 500),nn.Linear(500,  2))
model = model.cuda()
model.load_state_dict(torch.load(pathModel))
model = model.eval()


filter = model.features[0].weight.data.clone()
visTensor(filter.cpu(), ch=0, allkernels=False)
plt.axis('off')
plt.ioff()
plt.show()

In [ ]:
#covolucja z pierwszej denseleyer z pierwszego denseblockz
type(list(list(model.features[4].children())[0].children())[2]) == torch.nn.modules.conv.Conv2d

In [ ]:
model.features[0].weight.data.shape

In [ ]:
list(list(model.features[4].children())[4].children())[2]

In [ ]:
list(list(model.features[4].children())[0].children())[-1].weight.data.shape

In [ ]:

filter = list(list(model.features[4].children())[0].children())[-1].weight.data.clone()
visTensor(filter.cpu(), ch=0, allkernels=False)
plt.axis('off')
plt.ioff()
plt.show()

In [ ]:
list_weight = []
for i in model.features:
    if type(i) == torch.nn.modules.conv.Conv2d:
        list_weight.append(i.weight.data.clone())
    if type(i) in [torchvision.models.densenet._DenseBlock, torchvision.models.densenet._Transition]:
        for j in list(i.children()):
            if type(j) == torch.nn.modules.conv.Conv2d:
                list_weight.append(j.weight.data.clone())
            elif type(j) == torchvision.models.densenet._DenseLayer:
                for k in list(j.children()):
                    if type(k) == torch.nn.modules.conv.Conv2d:
                        list_weight.append(k.weight.data.clone())

In [61]:
name=1
for filter in list_weight:
    visTensor(filter.cpu(), ch=0, allkernels=False)
    plt.axis('off')
    plt.ioff()
    plt.savefig('filters/'+str(name))
    name+=1

C:\Users\micha\AppData\Local\Temp\ipykernel_10680\1798909486.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure( figsize=(nrow,rows) )
